# Make RGB NIRCam stamps showing slit footprint
We will be using the `trilogy` package to make RGB stamps and the shutter footprint regions.
The information we need to show the footprint and the source (optional) is all in the "shutters" files.


In [ ]:
from trilogy import trilogy
from astropy.io import fits
import astropy.wcs as wcs
import numpy as np
from astropy.table import Table
from astropy import table

from astropy.coordinates import SkyCoord
import astropy.units as u
import glob

import matplotlib
from matplotlib import pyplot as plt
import PIL  # Python Image Library
from PIL import Image

import os, sys

## Functions to make cutouts

In [ ]:
# helper functions
def roundint(x):
    return int(np.round(x))

def slices_extent(x, y, dx, dy=0):
    dy = dy or dx
    xlo = roundint(x-dx)
    xhi = roundint(x+dx+1)
    ylo = roundint(y-dy)
    yhi = roundint(y+dy+1)
    xslice = slice(xlo, xhi)
    yslice = slice(ylo, yhi)
    slices = yslice, xslice
    extent = xlo, xhi, ylo, yhi
    return slices, extent

In [ ]:
def make_stamp_with_shutters(imagesRGB, source_region, shutter_regions,
                             outfile='stamp.pdf', sourcemark=False, size_arcsec=3.0):
    """
    Make RGB stamp using trilogy and overplot the footprint of the shutters
    """
    
    ##################
    # MAKE RGB IMAGE #
    ##################
    
    # set luminosity of the noise in each channel
    noiselums = {'R': 0.2, 'G': 0.2, 'B': 0.2}
    
    old_stdout = sys.stdout # backup current stdout
    sys.stdout = open(os.devnull, "w") # avoid printing out stuff
    trilogy.Trilogy(infile = None, samplesize = 20000, stampsize = 20000, maxstampsize = 20000, \
                    deletetests = 1, deletefilters = 1, testfirst = 0, showwith = "PIL", \
                    mode = 'RGB', imagesorder = 'RGB', imagesRGB = imagesRGB, noiselums = noiselums, images = None, \
                    outname = 'color_image_temp', satpercent = 0.000, noiselum = 0.5, noisesig = 10, \
                    noisesig0 = 10, correctbias = 0, colorsatfac = 1, combine = 'sum', show = False).run()
    sys.stdout = old_stdout # reset old stdout
    
    # read in RGB image and delete png file
    im = Image.open('color_image_temp.png')
    im = im.transpose(method=Image.FLIP_TOP_BOTTOM)
    NIRCam_image = np.asarray(im)
    os.remove('color_image_temp.png')
    
    # read WCS from a single FITS file
    single_filename = imagesRGB['R'][0].split('[')[0] # remove '[1]' if present
    image_hdulist = fits.open(single_filename)
    image_wcs = wcs.WCS(image_hdulist[1].header, image_hdulist)
    
    #############################
    # MAKE IMAGE WITH FOOTPRINT #
    #############################
    
    # Define the image stamp region *centered on source*
    source_x, source_y = image_wcs.world_to_pixel(source_region.center)
    halfsize_arcsec = 0.5*size_arcsec*u.arcsec
    halfsize_pixels = int(halfsize_arcsec.to(u.deg).value / np.abs(image_wcs.pixel_scale_matrix[0,0]))
    slices, extent = slices_extent(source_x, source_y, halfsize_pixels)
    xlo, xhi, ylo, yhi = extent
    
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.imshow(NIRCam_image[slices], extent=extent, origin='lower')
    
    if sourcemark == True:
        #source_region.to_pixel(image_wcs).plot(fc='blue')
        ax.plot(source_x, source_y, 'o', mec='skyblue', mfc='None')
    
    for reg in shutter_regions:
        reg.to_pixel(image_wcs).plot(color='skyblue', alpha=1, lw=2)
    
    ax.axis('off')  # hide the axis coordinates, ticks, and labels
    fig.savefig(outfile, bbox_inches='tight', pad_inches=0.0)




## Make stamps

In [ ]:
# read in shutter footprints
import regions, pickle
with open('shutters_bluejay/shutters_regions.pkl', 'rb') as f:
    shutters = pickle.load(f)
print(len(shutters))


In [ ]:
# loop through the list of shutters
for s in shutters:

    # read shutters into regions objects
    reg_source = regions.Regions.parse(s['source'], format='ds9')
    reg_source = reg_source[0] # list of regions with only one entry
    reg_aperture = regions.Regions.parse(s['AB_aperture'], format='ds9')
    
    # read FITS files with NIRCam cutouts
    imagesRGB = {"R": [f"v1.0_JWST_cutouts/{s['ID']}_F444W_cutout.fits[1]"], \
                 "G": [f"v1.0_JWST_cutouts/{s['ID']}_F200W_cutout.fits[1]"], \
                 "B": [f"v1.0_JWST_cutouts/{s['ID']}_F115W_cutout.fits[1]"]}
    
    # make footprint
    make_stamp_with_shutters(imagesRGB, reg_source, reg_aperture,
                             outfile=f"stamps/{s['ID']}.pdf", sourcemark=False, size_arcsec=4.0)

